# **Abrindo o site do `Google Maps` e adicionando um destino e abre rotas**

> **MVP**:

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

from time import sleep

import pulp
import itertools

In [2]:
def esta_na_aba_de_rotas():

    # validando se está na aba de rotas ou não
    xpath = '//button[@aria-label="Fechar rotas"]'

    # pegando o botão das rotas
    botao_rotas = driver.find_elements(By.XPATH, xpath)

    return len(botao_rotas) > 0

In [3]:
def adiciona_destino(endereco, num_caixa=1):

    # se não encontrou as rotas
    if not esta_na_aba_de_rotas():

        # componente da busca
        barra_vazia = driver.find_element(By.ID, 'searchboxinput')

        # limpando barra vazia
        barra_vazia.clear()

        # enviando endereço
        barra_vazia.send_keys(endereco)

        # dando enter
        barra_vazia.send_keys(Keys.RETURN)

    # encontrou o botão
    else:

        # xpath das caixas
        xpath = '//div[contains(@id, "directions-searchbox")]//input'

        # elementos de busca
        caixas = driver.find_elements(By.XPATH, xpath)

        # validando se a caixa está sendo mostrada
        caixas = [c for c in caixas if c.is_displayed()]

        # se for verdadeiro
        if len(caixas) >= num_caixa:

            # caixa de endereço
            caixa_endereco = caixas[num_caixa - 1]

            # limpando variáveis
            caixa_endereco.send_keys(Keys.CONTROL + 'a')

            # removendo tudo
            caixa_endereco.send_keys(Keys.DELETE)

            # mandando endereço que o usuário pediu
            caixa_endereco.send_keys(endereco)

            # dando enter
            caixa_endereco.send_keys(Keys.RETURN)

        else:
            print(f'não foi possível adicionar o endereço: {
                  len(caixas)} | {num_caixa}')

In [4]:
def abre_rotas():
    
    # criando o padrão de busca das rotas
    xpath = '//button[@data-value="Rotas"]'

    # esperando um tempinho à mais
    wait = WebDriverWait(driver, timeout=7)

    # pegando o botão das rotas
    botao_rotas = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))

    # clicando no botão
    botao_rotas.click()

    # garantindo que as rotas estejam aberta
    xpath = '//button[@aria-label="Fechar rotas"]'

    # esperando um tempinho à mais
    wait = WebDriverWait(driver, timeout=7)

    # pegando o botão das rotas
    botao_rotas = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))

In [5]:
def adicionar_caixa_destino():

    # padrão da caixa de adicionar destino
    xpath = '//span[text()="Adicionar destino"]'

    # aguardando
    wait = WebDriverWait(driver, timeout=7)

    # esperando estar visível
    wait.until(EC.visibility_of_element_located((By.XPATH, xpath)))

    # buscando o elemento
    botao_adiciona_destino = driver.find_element(By.XPATH, xpath)

    # clicando
    botao_adiciona_destino.click()


In [6]:
def seleciona_tipo_conducao(tipo_conducao='Carro'):

    # padrão de busca
    xpath = f'//img[@aria-label="{tipo_conducao}"]'

    # aguardando
    wait = WebDriverWait(driver, timeout=7)

    # buscando elemento
    botao_conducao = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))

    # clicando
    botao_conducao.click()


In [7]:
def retorna_tempo_total():

    # padrão de busca
    xpath = '//div[@id="section-directions-trip-0"]//div[contains(text(), "min")]'

    # aguardando
    wait = WebDriverWait(driver, timeout=7)

    # buscando elemento
    elemento_tempo = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))

    return int(elemento_tempo.text.replace(' min', ''))


In [8]:
def retorna_distancia_total():

    # padrão de busca
    xpath = '//div[@id="section-directions-trip-0"]//div[contains(text(), "km")]'

    # aguardando
    wait = WebDriverWait(driver, timeout=7)

    # buscando elemento
    elemento_distancia = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))

    return float(elemento_distancia.text.replace(' km', '').replace(',', '.'))


In [13]:
enderecos = [
    'Av. José Bonifácio, 245 - Farroupilha, Porto Alegre - RS, 90040-130',  # redenção
    'Av. Borges de Medeiros, 2035 - Menino Deus, Porto Alegre - RS, 90110-150',  # marinha
    'Av. Guaíba, 544 - Ipanema, Porto Alegre - RS, 91760-740',  # orla ipanema
    'Av. Padre Cacique, 2000 - Praia de Belas, Porto Alegre - RS, 90810-180',  # iberê
    'R. Dr. Salvador França, 1427 - Jardim Botânico, Porto Alegre - RS, 90690-000',  # jardim botänico
]

def gera_pares_distancia(enderecos):

    distancia_pares = {}

    driver.get('https://www.google.com/maps')

    adiciona_destino(enderecos[0], 1)
    abre_rotas()
    seleciona_tipo_conducao(tipo_conducao="Carro")

    for i, end1 in enumerate(enderecos):
        adiciona_destino(end1, 1)
        for j, end2 in enumerate(enderecos):
            if i != j:
                adiciona_destino(end2, 2)
                # tempo_par = retorna_tempo_total()
                distancia_par = retorna_distancia_total()
                distancia_pares[f'{i}_{j}'] = distancia_par
    
    return distancia_pares


In [14]:
def gera_otimizacao(enderecos, distancia_pares):

    def distancia(end1, end2):
        return distancia_pares[f'{end1}_{end2}']
    
    prob = pulp.LpProblem('TSP', pulp.LpMinimize)

    x = pulp.LpVariable.dicts('x', [(i, j) for i in range(len(enderecos)) for j in range(len(enderecos)) if i != j], cat='Binary')

    prob += pulp.lpSum([distancia(i, j) * x[(i, j)] for i in range(len(enderecos)) for j in range(len(enderecos)) if i != j])

    # passar apenas uma vez
    for  i in range(len(enderecos)):
        prob += pulp.lpSum([x[(i, j)] for j in range(len(enderecos)) if i != j]) == 1
        prob += pulp.lpSum([x[(j, i)] for j in range(len(enderecos)) if i != j]) == 1

    # evitar subturs
    for k in range(len(enderecos)):
        for s in range(2, len(enderecos)):
            for subset in itertools.combinations([i for i in range(len(enderecos)) if i != k], s):
                prob += pulp.lpSum([x[(i, j)] for i in subset for j in subset if i != j]) <= len(subset) - 1

    prob.solve(pulp.PULP_CBC_CMD())

    # for i in range(len(enderecos)):
    #     for j in range(len(enderecos)):
    #         if i != j:
    #             print(i, j, x[(i, j)].value())
    
    solucao = []
    cidade_inicial = 0
    proxima_cidade = cidade_inicial
    while True:
        for j in range(len(enderecos)):
            if j != proxima_cidade and x[(proxima_cidade, j)].value() == 1:
                solucao.append((proxima_cidade, j))
                proxima_cidade = j
                break
        if proxima_cidade == cidade_inicial:
            break

    print('Rota: ')
    for i in range(len(solucao)):
        print(solucao[i][0], ' ->> ', solucao[i][1])
    
    return solucao

# evitando erros de encontrar drivers que abre o navegador no computador
service = Service(ChromeDriverManager().install())

# instanciando o driver
driver = webdriver.Chrome()

# deixando uma espera implícita
driver.implicitly_wait(7)

# abrindo o navegador
driver.get('https://www.google.com/maps')

solucao = gera_otimizacao(enderecos, gera_pares_distancia(enderecos))

Rota: 
0  ->>  1
1  ->>  3
3  ->>  2
2  ->>  4
4  ->>  0


In [15]:
def mostra_rota_otimizada(enderecos, solucao):
    driver.get('https://www.google.com/maps')

    adiciona_destino(enderecos[0], 1)
    abre_rotas()

    for i in range(len(solucao)):
        adiciona_destino(enderecos[solucao[i][0]], i+1)
        adicionar_caixa_destino()
    
    adiciona_destino(enderecos[solucao[0][0]], len(enderecos) + 1)

mostra_rota_otimizada(enderecos, solucao)

In [ ]:
if __name__ == '__main__':

    # evitando erros de encontrar drivers que abre o navegador no computador
    service = Service(ChromeDriverManager().install())

    # instanciando o driver
    driver = webdriver.Chrome()

    # deixando uma espera implícita
    driver.implicitly_wait(7)

    # abrindo o navegador
    driver.get('https://www.google.com/maps')

    # litagem de endereços
    enderecos = [
        'Av. José Bonifácio, 245 - Farroupilha, Porto Alegre - RS, 90040-130',  # redenção
        'Av. Borges de Medeiros, 2035 - Menino Deus, Porto Alegre - RS, 90110-150',  # marinha
        'Av. Guaíba, 544 - Ipanema, Porto Alegre - RS, 91760-740',  # orla ipanema
        'Av. Padre Cacique, 2000 - Praia de Belas, Porto Alegre - RS, 90810-180',  # iberê
        'R. Dr. Salvador França, 1427 - Jardim Botânico, Porto Alegre - RS, 90690-000',  # jardim botänico
    ]

    # adicionando destino
    adiciona_destino(enderecos[0], 1)

    # abrindo rotas
    abre_rotas()

    # adicionando as rotas
    adiciona_destino(enderecos[0], 1)
    adiciona_destino(enderecos[1], 2)

    # adicionando mais uma caixa de destino
    adicionar_caixa_destino()

    # adicionando rotas
    adiciona_destino(enderecos[2], 3)

    # adicionando mais uma caixa de destino
    adicionar_caixa_destino()

    # adicionando rotas
    adiciona_destino(enderecos[3], 4)

    # selecionando o tipo de condução
    seleciona_tipo_conducao(tipo_conducao="Carro")

    print(retorna_tempo_total())
    print(retorna_distancia_total())

    # esperando processamento
    # sleep(600)

47
24.2
